In [3]:
import psycopg2
import pandas as pd

# Prepare the Data

Run `docker-compose up` to get Postgres up and running.

In [4]:
credentials = "dbname='astronomy' user='user' host='localhost' password='pass'"
conn = psycopg2.connect(credentials)
cur = conn.cursor()

cur.execute("""
DROP TABLE IF EXISTS Planet;
DROP TABLE IF EXISTS Star;

CREATE TABLE Star (
  kepler_id INTEGER NOT NULL,
  t_eff INTEGER NOT NULL,
  radius FLOAT NOT NULL,
  PRIMARY KEY (kepler_id)
);

CREATE TABLE Planet (
  kepler_id INTEGER NOT NULL REFERENCES Star(Kepler_ID),
  koi_name VARCHAR(20) NOT NULL,
  kepler_name VARCHAR(20),
  status VARCHAR(20) NOT NULL,
  period FLOAT NOT NULL,
  radius FLOAT NOT NULL,
  t_eq INTEGER NOT NULL,
  PRIMARY KEY (koi_name)
);


COPY Star (kepler_id, t_eff, radius)
  FROM '/data/data3/stars.csv' CSV;

COPY Planet (kepler_id, koi_name, kepler_name, status, period, radius, t_eq)
  FROM '/data/data3/planets.csv' CSV;
""")
conn.commit()

# Task 1
Write a query that returns the radius of each star and planet pair whose radii
have a ratio greater than the Sun-to-Earth radius ratio. Order the results in
descending order based on the stellar radii. Use `sun_radius` and
`planet_radius` as attribute aliases for the star and planet radii.

For this problem you will have to join the two tables to find all planets
belonging to a given star and use a condition to select those results which
fulfill the size requirement above.

In [ ]:
pd.read_sql("""
SELECT
  Star.radius as sun_radius,
  Planet.radius as planet_radius
FROM Star, Planet
WHERE
  Star.kepler_id = Planet.kepler_id AND
  Star.radius > Planet.radius
ORDER BY Star.radius DESC;
""", conn)

# Tear down

In [ ]:
cur.close()
conn.close()